In [1]:
import os 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt


# import cv2
import neuroseries as nts

import bk.load
import bk.compute
import bk.plot

In [2]:
import scipy.stats

In [3]:
import matplotlib as mpl
# COLOR = 'white'
# mpl.rcParams['text.color'] = COLOR
# mpl.rcParams['axes.labelcolor'] = COLOR
# mpl.rcParams['axes.edgecolor'] = COLOR
# mpl.rcParams['xtick.color'] = COLOR
# mpl.rcParams['ytick.color'] = COLOR
# mpl.rcParams['figure.facecolor'] = '#282C34'
# mpl.rcParams['axes.facecolor'] = '#282C34'
mpl.rcParams['svg.fonttype'] = 'none'
# %matplotlib qt

In [4]:
def DLC_pos(filtered = True,force_reload = False, save = False):
    """
    Load position from DLC files (*.h5) and returns it as a nts.TsdFrame
    """
    files = os.listdir()
    if ('positions.h5' in files) and (force_reload == False):
        data = pd.read_hdf('positions.h5')
        pos = nts.TsdFrame(data)
        return pos

    for f in files:
        if filtered and f.endswith('filtered.h5'): 
            filename = f
            break
        if not filtered and not f.endswith('filtered.h5') and f.endswith('.h5'):
            filename = f
            break

    data = pd.read_hdf(filename)
    data = data[data.keys()[0][0]]

    TTL = bk.load.digitalin('digitalin.dat')[0,:]
    tf = bk.compute.TTL_to_times(TTL)

    if len(tf)>len(data):
        tf = np.delete(tf,-1)

#     data.index = tf * 1_000_000
    
    if save:
        data.to_hdf('positions.h5','pos')
        
    pos = nts.TsdFrame(data)
    return pos

In [12]:
def compute_freezing(path):
    os.chdir(path)
    
    camera_pixel_cm = 0.1856
    exp,shock,tone = bk.compute.intervals_exp()
    delay = exp.start.values[0]
    pos = DLC_pos()

    pos.index = (np.linspace(0,len(pos)*1/30,len(pos))*1_000_000) - delay
    
    speed = bk.compute.speed(pos,60,columns_to_drop=['end-tail','base-tail'])
    speed = speed * camera_pixel_cm
    speed.columns.values
    freezing_intervals = bk.compute.freezing_intervals(speed,3,'multiple_speed',clean= True)
    
    plt.plot(np.mean(speed.as_units('s'),1),'red')
    bk.plot.intervals(exp-delay,'blue',alpha = 1,ymin = 0.9,ymax = 1)
    bk.plot.intervals(tone-delay,'green',alpha = 0.3)
    bk.plot.intervals(shock-delay,'yellow',alpha= 1)
    
#     print(tone.start[0]-delay)
#     print(shock.as_units('s')/1_000_000)
    bk.plot.intervals(freezing_intervals,'red',alpha = 1,ymin = 0.8,ymax = 0.9)
    plt.ylim([0,50])
    plt.legend(['Speed'])
    plt.show()
    
    print((exp.iloc[0].end - exp.iloc[0].start) / 1_000_000)
    
    return freezing_intervals

In [13]:
def pourcent_freezing(freezing_intervals,interval): 
    freezing_intervals_in_interval = freezing_intervals.intersect(interval)
    total_time_freezing = np.sum(freezing_intervals_in_interval.end-freezing_intervals_in_interval.start)
    total_time = interval.end-interval.start
    pourcentage = (total_time_freezing/total_time)*100
    return pourcentage

In [16]:
base_path = '/mnt/electrophy/Behavior/Eleonore/classical_fear_conditinning_decorr/'
os.path.join(base_path,'indexing.csv')

indexing = pd.read_csv(os.path.join(base_path,'indexing.csv'))
exp = ['fear','context','tone']
rats = np.unique(indexing.rat)

freezing = {}
for rat in rats:
    plt.figure(figsize=(12,4))
    all_freezing_intervals = {}
    for i,e in enumerate(exp,1):
        plt.subplot(3,1,i)
        print(rat,e)
        local_path = indexing.relative_path[(indexing.rat == rat) & (indexing.exp == e)].values[0]
        freezing_intervals = compute_freezing(os.path.join(base_path,local_path))
        all_freezing_intervals.update({e:freezing_intervals})
        freezing.update({rat:all_freezing_intervals})
        plt.suptitle(f'Rat : {rat}')
        
        plt.title(f'Exp : {e}')
        plt.tight_layout()
    plt.savefig(f'/home/billel/Documents/figs-ele/rat{rat}.png',dpi = 300)
    plt.savefig(f'/home/billel/Documents/figs-ele/rat{rat}.svg')

26 fear
600.6916
26 context
600.38995
26 tone
600.5643
27 fear
600.6955
27 context
600.40135
27 tone
600.5754
28 fear
600.68875
28 context
600.39535
28 tone
600.57175
29 fear
600.68355
29 context
600.3903
29 tone
600.56495
30 fear
600.6985
30 context
600.3832
30 tone
600.56895
31 fear
600.6821
31 context
600.38455
31 tone
600.5679


In [17]:
bins = np.arange(0,600,30)
rats = np.unique(indexing.rat)
experience = ['fear','context','tone']
axes = ['A','B','C']
fig,ax = plt.subplot_mosaic('''A
                               B
                               C''',
                            figsize = (12,8))

for e,a in zip(experience,axes):
    print(a)
    os.chdir(os.path.join(base_path,indexing.relative_path[indexing.exp == e].iloc[0]))
    print(os.path.join(base_path,indexing.relative_path[indexing.exp == e].iloc[0]))
    exp,shock,tone = bk.compute.intervals_exp()
    plt.sca(ax[a])
    bk.plot.intervals(shock-exp.start[0],'yellow',alpha=0.5)
    bk.plot.intervals(tone-exp.start[0],'green',alpha = 0.5)

    mean_exp = []
    for r in rats:
        ax[a].set_title(e)
        t_f = [pourcent_freezing(freezing[r][e],nts.IntervalSet(b,b+30,time_units='s')) for b in bins]
        mean_exp.append(t_f)
        ax[a].plot(bins+15,t_f)
        plt.sca(ax[a])
        plt.legend(rats)
    m = np.mean(mean_exp,0).ravel()
    std = scipy.stats.sem(mean_exp,0).ravel()
    
    ax[a].plot(bins+15,m,'k-')
    ax[a].fill_between(bins+15,m-std,m+std,alpha = 0.2)
    ax[a].set_ylim([0,100])
    
    
    ax[a].set_ylabel('% Freezing')
    ax[a].set_xlabel('Time (s)')
plt.tight_layout()
plt.savefig(f'/home/billel/Documents/figs-ele/all.png',dpi = 300)
plt.savefig(f'/home/billel/Documents/figs-ele/all.svg')

A
/mnt/electrophy/Behavior/Eleonore/classical_fear_conditinning_decorr/Rat26/Rat26-210305/Rat26_210305_100504
B
/mnt/electrophy/Behavior/Eleonore/classical_fear_conditinning_decorr/Rat26/Rat26-210305/Rat26_210305_140239
C
/mnt/electrophy/Behavior/Eleonore/classical_fear_conditinning_decorr/Rat26/Rat26-210305/Rat26_210305_150351
